In [72]:
import warnings
warnings.filterwarnings("ignore")

In [73]:
import pandas as pd
import re

In [74]:
def csv_to_dataframe(file_path):
    return pd.read_csv(file_path)

In [75]:
def dataframe_columns_to_dict(df_apple_volume_mapper, key_column, value_column):
    return df_apple_volume_mapper.set_index(key_column)[value_column].to_dict()

In [76]:
file_path = '../Data/fitted_ellipse_parameter.csv'  
df_fitted_ellipse_per_view = csv_to_dataframe(file_path)

In [77]:
file_path = '../Data/apple_volume_mapper.csv'  
df_apple_volume_mapper = csv_to_dataframe(file_path)

In [78]:
apple_volumn_mapper_dict = dataframe_columns_to_dict(df_apple_volume_mapper, 'Apple No.', 'Volume')

In [79]:
data = {
    'apple_label': [],
    'semi_major_axis': [],
    'semi_minor_axis': [],
    'area': [],
    'perimeter': [],
    'volume': [],
}
avg_fitted_ellipse_parameter = pd.DataFrame(data)

In [80]:
for i in range(1, 201):
    
    avg_semi_major_axis = 0
    avg_semi_minor_axis = 0
    avg_perimeter = 0
    avg_area = 0
    volume = 0
    count_views = 0
    
    for index, row in df_fitted_ellipse_per_view.iterrows():
        apple_idx = re.findall(r'\d+', row['apple_label'])
        if int(apple_idx[0]) == i:
            count_views +=1
            avg_semi_major_axis += row['semi_major_axis']
            avg_semi_minor_axis += row['semi_minor_axis']
            avg_perimeter += row['perimeter']
            avg_area += row['area']
            apple_label = float(re.sub(r'\D', '', row['apple_label']))
            
    if count_views == 0:
        continue
    avg_semi_major_axis /= count_views
    avg_semi_minor_axis /= count_views
    avg_perimeter /= count_views
    avg_area /= count_views
    volume = apple_volumn_mapper_dict[apple_label]

    new_row = {'apple_label': i, 'semi_major_axis': avg_semi_major_axis, 'semi_minor_axis': avg_semi_minor_axis, 'area': avg_area, 'perimeter': avg_perimeter, 'volume': volume}
    new_row_df = pd.DataFrame([new_row])
    avg_fitted_ellipse_parameter = pd.concat([avg_fitted_ellipse_parameter, new_row_df], ignore_index=True)

In [85]:
avg_fitted_ellipse_parameter = avg_fitted_ellipse_parameter.round(2)
avg_fitted_ellipse_parameter['apple_label'] = avg_fitted_ellipse_parameter['apple_label'].astype(int)

In [86]:
avg_fitted_ellipse_parameter.to_csv('../Data/avg_fitted_ellipse_parameter_and_true_volume.csv', index=False)